<a href="https://colab.research.google.com/github/SauravBedse223/Data-Pipeline-Creation-for-YouTube-and-Instagram-Content/blob/main/Data_Pipeline_Creation_for_YouTube_and_Instagram_Content.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install youtube_dl instaloader SpeechRecognition pydub pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 29.2 MB/s eta 0:00:00
  Created wheel for instaloader: filename=instaloader-4.11-py3-none-any.whl size=65967 sha256=23058476f46e6563e947a4cdf119acf46a7353ce7e9b8de221f13d65ca9f2a55
  Stored in directory: /root/.cache/pip/wheels/59/74/a0/a643055404b7f51b588926301ce74f4ad195e5d45cde82e79d
Successfully built instaloader


In [28]:
import instaloader

# Initialize Instaloader
L = instaloader.Instaloader()

# Log in to Instagram
username = 'saurav_bedse_prt'
password = '********'
L.login(username, password)

# Save the session
L.save_session_to_file()


Saved session to /root/.config/instaloader/session-saurav_bedse_prt.


In [22]:
import youtube_dl
import instaloader
import speech_recognition as sr
from pydub import AudioSegment
import pandas as pd
import os


In [23]:
# Function to download audio from YouTube videos
def download_youtube_audio(url, output_path):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'outtmpl': output_path,
    }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

In [24]:
# Function to download video from Instagram
def download_instagram_video(url, output_path, username):
    L = instaloader.Instaloader()
    # Load the saved session
    session_file = f"/root/.config/instaloader/session-{username}"

    if not os.path.exists(session_file):
        raise FileNotFoundError(f"Session file {session_file} not found.")

    L.load_session_from_file(username, session_file)

    # Extract shortcode from URL
    shortcode = url.split('/')[4]
    post = instaloader.Post.from_shortcode(L.context, shortcode)
    L.download_post(post, target=output_path)

In [25]:
# Function to transcribe audio
def transcribe_audio(audio_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio_data = recognizer.record(source)
        try:
            return recognizer.recognize_google(audio_data)
        except sr.UnknownValueError:
            return "Transcription failed."
        except sr.RequestError as e:
            return f"Could not request results from Google Web Speech API; {e}"

In [26]:
# Convert audio file to WAV format (if necessary)
def convert_to_wav(input_audio_path, output_audio_path):
    audio = AudioSegment.from_file(input_audio_path)
    audio.export(output_audio_path, format="wav")

In [27]:
def main():
    # Load the dataset
    file_path = "/content/drive/MyDrive/Colab Notebooks/Copy of AI Project Data Training Sheet - Data Sheet.csv"
    data = pd.read_csv(file_path)

    # Ensure output directories exist
    os.makedirs("downloads", exist_ok=True)
    os.makedirs("audios", exist_ok=True)

    # Username for session loading
    username = 'saurav_bedse_prt'

    # Iterate through each row in the dataset
    for index, row in data.iterrows():
        url = row["Reference URL"]
        video_type = "youtube" if "youtube" in url.lower() else "instagram"
        video_id = url.split('/')[-1]
        video_path = f"downloads/{video_type}_{video_id}.mp4"
        audio_path = f"audios/{video_type}_{video_id}.wav"

        # Download the video and extract audio
        if video_type == "youtube":
            try:
                download_youtube_audio(url, video_path)
            except Exception as e:
                print(f"Failed to download YouTube video {url}: {e}")
                continue
        else:
            try:
                download_instagram_video(url, "downloads", username)
            except Exception as e:
                print(f"Failed to download Instagram video {url}: {e}")
                continue

        # Check if the file was downloaded correctly
        if not os.path.exists(video_path):
            video_files = [file for file in os.listdir("downloads") if file.endswith('.mp4')]
            if video_files:
                video_path = os.path.join("downloads", video_files[0])
            else:
                print(f"No video file found for {url}")
                continue

        # Convert to WAV format if necessary
        if not video_path.endswith(".wav"):
            try:
                convert_to_wav(video_path, audio_path)
            except Exception as e:
                print(f"Failed to convert video to WAV {video_path}: {e}")
                continue

        # Transcribe the audio
        try:
            transcription = transcribe_audio(audio_path)
        except Exception as e:
            print(f"Failed to transcribe audio {audio_path}: {e}")
            transcription = "Transcription failed."

        # Update the dataset
        data.at[index, "Body"] = transcription

    # Save the updated dataset
    data.to_csv("/content/drive/MyDrive/Colab Notebooks/transcribed_data.csv", index=False)
    print("Transcription complete and saved to transcribed_data.csv")

if __name__ == "__main__":
    main()

Loaded session from /root/.config/instaloader/session-saurav_bedse_prt.
downloads/2024-03-29_15-05-21_UTC.jpg exists [Multiple creators stole from …] unchanged downloads/2024-03-29_15-05-21_UTC.mp4 exists json 
Loaded session from /root/.config/instaloader/session-saurav_bedse_prt.
downloads/2024-04-20_20-15-46_UTC.jpg exists [Comment “CTA” for the doc…] unchanged downloads/2024-04-20_20-15-46_UTC.mp4 exists json 
Loaded session from /root/.config/instaloader/session-saurav_bedse_prt.
downloads/2023-09-27_14-12-25_UTC.jpg exists [$187,348 Social Media Strateg…] unchanged downloads/2023-09-27_14-12-25_UTC.mp4 exists json 
Loaded session from /root/.config/instaloader/session-saurav_bedse_prt.
downloads/2023-12-30_18-31-15_UTC.jpg exists [comment “wallet”   #socialmed…] unchanged downloads/2023-12-30_18-31-15_UTC.mp4 exists json 
Loaded session from /root/.config/instaloader/session-saurav_bedse_prt.
downloads/2023-09-20_11-44-49_UTC.jpg exists [🤖Algorithm Explained…  - - - …] unchanged 

JSON Query to api/v1/media/3285451686787142768/info/: HTTP error code 401. [retrying; skip with ^C]
JSON Query to api/v1/media/3285451686787142768/info/: HTTP error code 401. [retrying; skip with ^C]
Unable to fetch high-quality video version of <Post C2YRObtp9Rw>: JSON Query to api/v1/media/3285451686787142768/info/: HTTP error code 401.


downloads/2024-01-21_22-34-10_UTC.mp4 json 
Loaded session from /root/.config/instaloader/session-saurav_bedse_prt.


JSON Query to graphql/query: HTTP error code 401. [retrying; skip with ^C]
JSON Query to graphql/query: HTTP error code 401. [retrying; skip with ^C]


Failed to download Instagram video https://www.instagram.com/reel/C3sMEhAL6L-/?igsh=MTJ6bG4zanRsZjc2ZA==: JSON Query to graphql/query: HTTP error code 401.
Loaded session from /root/.config/instaloader/session-saurav_bedse_prt.


...

Failed to download Instagram video https://www.instagram.com/reel/C6oWzi8i4A_/?igsh=MXZjc2R6Y3djemVrcA==: JSON Query to graphql/query: HTTP error code 401.
Loaded session from /root/.config/instaloader/session-saurav_bedse_prt.


JSON Query to graphql/query: HTTP error code 401. [retrying; skip with ^C]
JSON Query to graphql/query: HTTP error code 401. [retrying; skip with ^C]


Failed to download Instagram video https://www.instagram.com/reel/C2C1F7ACy9B/?igsh=d2NjZnZsY283MHF3: JSON Query to graphql/query: HTTP error code 401.
Loaded session from /root/.config/instaloader/session-saurav_bedse_prt.


JSON Query to graphql/query: HTTP error code 401. [retrying; skip with ^C]
JSON Query to graphql/query: HTTP error code 401. [retrying; skip with ^C]


Failed to download Instagram video https://www.instagram.com/reel/C0eV7_fCYKs/?igsh=MXRsZGszODRobnJ1MA==: JSON Query to graphql/query: HTTP error code 401.
Transcription complete and saved to transcribed_data.csv
